In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import dataloader as dl
from network import Network

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
model = Network().to(device)
print(model)

Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
    (5): LogSoftmax(dim=1)
  )
)


In [4]:
train_dataset = dl.MNISTData("./data/", True)
test_dataset = dl.MNISTData("./data/", False)
train_data = DataLoader(train_dataset, 64, True)
test_data = DataLoader(test_dataset, 64)

In [5]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        losses += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch completed. Avg loss: {losses/len(dataloader)}")     

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 10
for epoch in range(1, epochs+1):
    print(f"---- Epoch {epoch} is starting ----")
    train(train_data, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)

---- Epoch 1 is starting ----
Epoch completed. Avg loss: 0.014789615757763386
Test Error: 
 Accuracy: 96.5%, Avg loss: 0.114876 

---- Epoch 2 is starting ----
Epoch completed. Avg loss: 0.013910792768001556
Test Error: 
 Accuracy: 96.6%, Avg loss: 0.114553 

---- Epoch 3 is starting ----
Epoch completed. Avg loss: 0.013111294247210026
Test Error: 
 Accuracy: 96.6%, Avg loss: 0.114637 

---- Epoch 4 is starting ----
